In [1]:
import pydicom
import os,re
import numpy as np
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from pydicom.dataset import Dataset, FileDataset
import tempfile
import SimpleITK as sitk
import math
from ipywidgets import interact, fixed
from skimage.transform import resize
from IPython.display import clear_output
from scipy.ndimage import zoom
import scipy.misc
from sys import platform
import shutil

from MyProcessDicom import *
from RegistrationMethod import *
from ServerDicomIO import *
platform

'win32'

In [2]:
import importlib
import sys
sys.path.append(r'..\FRAPPE')
import DB
importlib.reload(DB)
from DB import DB

# process list
# first in TP as reference
caselist = [
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'side':'L'}
]

# getdcmpath('9911265',0,'L'),getdcmpath('9911265',1,'R'),getdcmpath('9911265',8,'R')

In [3]:
#split seqs for a case
getFileFromDB(caselist)

********  tp 0 ************
ahaknee0tp29  exist
dcm path for TPid 0 Z:/OAI Baseline/gc0/9404954/10728107


FileNotFoundError: [WinError 53] The network path was not found: 'Z:/OAI Baseline/gc0/9404954/10728107'

In [4]:
def do_registration(casei, regis_fun):
    pi = casei['pid']
    side = casei['side']
    precasepath = split_seq_path + pi + side + '/'
    regtp = casei['TP']
    if len(regtp) < 2:
        print(pi, 'not enough TPs')
        return
    tp1 = regtp[0]
    print("Fixed Image %d" % tp1)
    tp1_casepath = precasepath + 'TP0/'
    fixed_image = get_dicom_series(tp1_casepath)
    
    icafesavepath = icafepath+'0_P'+pi+side+'_U'+'/'
    if not os.path.exists(icafesavepath):
        print('Init icafe dir')
        os.mkdir(icafesavepath)
    
    # isotropic resolution
    img1 = resample_image(fixed_image)
    sitk.WriteImage(sitk.Cast(img1, sitk.sitkInt16), icafesavepath + 'TH_0_P' + pi + side + '_U.tif')
    print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_U.tif')
    sitk.WriteImage(sitk.Cast(img1, sitk.sitkInt16), icafesavepath + 'TH_0_P'+  pi + side + '_US100.tif')
    print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_US100.tif')
    
    fixed_ret, resampled_ret = [], []
    for tp2 in regtp[1:]:
        #if tp2 != 1: continue
        print("************************************************")
        print("Processing %d......" % tp2)
        tp2_casepath = getdcmpath(pi,tp2,side) 
        if tp2_casepath is None:
            print('cannot find dcm path for TPid', tp2)
            continue
        
        tp2_casepath = precasepath + 'TP' + str(tp2) + '/'
        print("reading image......", pi + 'TP' + str(tp2))
        moving_image = get_dicom_series(tp2_casepath)
        fixed_image = preprocess(fixed_image)
        moving_image = preprocess(moving_image)
        show_images(fixed_image, moving_image)
        
        transform = regis_fun(fixed_image = fixed_image, moving_image = moving_image, vis=False)
        if tp2==10:
            SEQ = 'S109'
        else:
            SEQ = 'S10'+str(tp2)
        sitk.WriteTransform(transform, transform_path + 'P'+  pi + side + 'U' + SEQ + '.tfm')
        tmp_img = transform_img(fixed_image, moving_image, transform)

#         sitk.WriteImage(sitk.Cast(exqimg, sitk.sitkInt16), icafesavepath + 'TH_0_P'+  pi + side + '_U' + SEQ + '.tif')
#         print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_U' + SEQ + '.tif')
        fixed_ret.append(fixed_image)
        resampled_ret.append(tmp_img)
    return fixed_ret, resampled_ret

In [5]:
caselist[0]['TP']

[0, 1, 2, 3, 4, 5, 6, 8, 10]

In [6]:
for casei in caselist:
    ret = do_registration(casei, affine_registration)
#     if ret is not None:
#         fixed_image, moving_resampled = ret
#         interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), 
#                  fixed = fixed(fixed_image), moving=fixed(moving_resampled));

Fixed Image 0
coronal split_seq_OAI/9404954L/TP0/001.dcm
Init icafe dir


FileNotFoundError: [WinError 53] The network path was not found: 'Y:/iCafe/result/OAIMTP/0_P9404954L_U/'

In [23]:
show_mixed_images()

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [26]:
# FFD transform(Free Form Deformation)
def bspline_intra_modal_registration(fixed_image, moving_image, fixed_image_mask=None, fixed_points=None, 
                                     moving_points=None, vis=True):

    registration_method = sitk.ImageRegistrationMethod()
    
    # Determine the number of BSpline control points using the physical spacing we want for the control grid. 
    grid_physical_spacing = [50.0, 50.0, 50.0] # A control point every 50mm
    image_physical_size = [size*spacing for size,spacing in zip(fixed_image.GetSize(), fixed_image.GetSpacing())]
    mesh_size = [int(image_size/grid_spacing + 0.5) \
                 for image_size,grid_spacing in zip(image_physical_size,grid_physical_spacing)]

    initial_transform = sitk.BSplineTransformInitializer(image1 = fixed_image, 
                                                         transformDomainMeshSize = mesh_size, order=3)    
    registration_method.SetInitialTransform(initial_transform)
        
    registration_method.SetMetricAsMeanSquares()
    # Settings for metric sampling, usage of a mask is optional. When given a mask the sample points will be 
    # generated inside that region. Also, this implicitly speeds things up as the mask is smaller than the
    # whole image.
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    if fixed_image_mask:
        registration_method.SetMetricFixedMask(fixed_image_mask)
    
    # Multi-resolution framework.            
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5, numberOfIterations=100)
    
    # If corresponding points in the fixed and moving image are given then we display the similarity metric
    # and the TRE during the registration.
    if vis:
        if fixed_points and moving_points:
            registration_method.AddCommand(sitk.sitkStartEvent, rc.metric_and_reference_start_plot)
            registration_method.AddCommand(sitk.sitkEndEvent, rc.metric_and_reference_end_plot)
            registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rc.metric_and_reference_plot_values(registration_method, fixed_points, moving_points))
        else:
            registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
            registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
            registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations) 
            registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))
    
    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), sitk.Cast(moving_image, sitk.sitkFloat32))
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    return final_transform

def generate_random_pointset(image, image2, num_points):
    """
    Generate a random set (uniform sample) of points in the given image's domain.
    
    Args:
        image (SimpleITK.Image): Domain in which points are created.
        num_points (int): Number of points to generate.
        
    Returns:
        A list of points (tuples).
    """
    # Continous random uniform point indexes inside the image bounds.
    point_indexes = np.multiply(np.tile(image.GetSize(), (num_points, 1)), 
                                np.random.random((num_points, image.GetDimension())))
    pointset_list = point_indexes.tolist()
    # Get the list of physical points corresponding to the indexes.
    return [image.TransformContinuousIndexToPhysicalPoint(point_index) \
            for point_index in pointset_list], [image2.TransformContinuousIndexToPhysicalPoint(point_index) \
            for point_index in pointset_list]

In [41]:
for casei in caselist:
    fixed_image, moving_image = do_registration(casei, bspline_intra_modal_registration)

[384 384  75] [384, 384, 309]
save to icafe path D:\iCafe\result\OAIMTP/0_P9404954R_U/TH_0_P9404954R_U.tif
save to icafe path D:\iCafe\result\OAIMTP/0_P9404954R_U/TH_0_P9404954R_US100.tif
ahaknee12tp29  exist
coronal split_seq_OAI/9404954R/TP1/001.dcm
reading image...... 9404954TP1
Final metric value: 417.68280532659725
Optimizer's stopping condition, LBFGSBOptimizerv4: Function tolerance reached after 60 iterations. The relative reduction of the cost function <= 2.22045e-09 = CostFunctionConvergenceFactor (1e+07) * machine precision (2.22045e-16).
save to icafe path D:\iCafe\result\OAIMTP/0_P9404954R_U/TH_0_P9404954R_US101.tif
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
coronal split_seq_OAI/9404954R/TP3/076.dcm
reading image...... 9404954TP3
Final metric value: 1755.681177331056
Optimizer's stopping condition, LBFGSBOptimizerv4: Function tolerance reached after 37 iterations. The relative reduction of the cost function <= 2.22045e-09 = CostFunctionConver

In [38]:
moving_resampled = sitk.Resample(moving_image, fixed_image, ffd_transform, sitk.sitkBSpline, 0.0, moving_image.GetPixelID())

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_resampled));

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [27]:
# DEMONS
def demons_registration(fixed_image, moving_image, fixed_points = None, moving_points = None, vis=True):
    registration_method = sitk.ImageRegistrationMethod()

    # Create initial identity transformation.
    transform_to_displacment_field_filter = sitk.TransformToDisplacementFieldFilter()
    transform_to_displacment_field_filter.SetReferenceImage(fixed_image)
    # The image returned from the initial_transform_filter is transferred to the transform and cleared out.
    initial_transform = sitk.DisplacementFieldTransform(transform_to_displacment_field_filter.Execute(sitk.Transform()))
    
    # Regularization (update field - viscous, total field - elastic).
    initial_transform.SetSmoothingGaussianOnUpdate(varianceForUpdateField=0.0, varianceForTotalField=2.0) 
    
    registration_method.SetInitialTransform(initial_transform)

    registration_method.SetMetricAsDemons(10) #intensities are equal if the difference is less than 10HU
        
    # Multi-resolution framework.            
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[8,4,0])    

    registration_method.SetInterpolator(sitk.sitkLinear)
    # If you have time, run this code as is, otherwise switch to the gradient descent optimizer    
    #registration_method.SetOptimizerAsConjugateGradientLineSearch(learningRate=1.0, numberOfIterations=20, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=20, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # If corresponding points in the fixed and moving image are given then we display the similarity metric
    # and the TRE during the registration.
    if vis:
        if fixed_points and moving_points:
            registration_method.AddCommand(sitk.sitkStartEvent, rc.metric_and_reference_start_plot)
            registration_method.AddCommand(sitk.sitkEndEvent, rc.metric_and_reference_end_plot)        
            registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rc.metric_and_reference_plot_values(registration_method, fixed_points, moving_points))
        else:
            registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
            registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
            registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations) 
            registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), sitk.Cast(moving_image, sitk.sitkFloat32))
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    return final_transform  

In [28]:
def our_pipeline(casei):
    pi = casei['pid']
    side = casei['side']
    precasepath = split_seq_path + pi + side + '/'
    regtp = casei['TP']
    if len(regtp) < 2:
        print(pi, 'not enough TPs')
        return
    tp1 = regtp[0]
    print("Fixed Image %d" % tp1)
    tp1_casepath = precasepath + 'TP0/'
    fixed_image = get_dicom_series(tp1_casepath)
    
    icafesavepath = icafepath+'0_P'+pi+side+'_U'+'/'
    if not os.path.exists(icafesavepath):
        print('Init icafe dir')
        os.mkdir(icafesavepath)
    
    # isotropic resolution
    
    fixed_image = resample_image(fixed_image)
    img1 = fixed_image
    sitk.WriteImage(sitk.Cast(fixed_image, sitk.sitkInt16), icafesavepath + 'TH_0_P' + pi + side + '_U.tif')
    print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_U.tif')
    sitk.WriteImage(sitk.Cast(fixed_image, sitk.sitkInt16), icafesavepath + 'TH_0_P'+  pi + side + '_US100.tif')
    print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_US100.tif')
    
#     fixed_image = img1
    fixed_image = preprocess(img1)
    fixed_ret, resampled_ret = [], []
    for tp2 in regtp[1:3]:
        #if tp2 != 1: continue
        print("************************************************")
        print("Processing %d..." % tp2)
        tp2_casepath = getdcmpath(pi,tp2,side) 
        if tp2_casepath is None:
            print('cannot find dcm path for TPid', tp2)
            continue
        
        tp2_casepath = precasepath + 'TP' + str(tp2) + '/'
        print("reading image......", pi + 'TP' + str(tp2))
        moving_image = get_dicom_series(tp2_casepath)
        img2 = resample_image(moving_image)
        moving_image = preprocess(img2)
        
        if tp2 <=8:
            transform = rigid_registration(fixed_image, moving_image, vis=False)
            img2 = sitk.Resample(empty_preprocess(img2), empty_preprocess(img1), transform, sitk.sitkBSpline, 0.0, empty_preprocess(img2).GetPixelID())
            moving_image = sitk.Resample(moving_image, fixed_image, transform, sitk.sitkBSpline, 0.0, moving_image.GetPixelID())
        
        ffd_transform = bspline_intra_modal_registration(fixed_image, moving_image, vis=False)
        exqimg = sitk.Resample(empty_preprocess(img2), empty_preprocess(img1), ffd_transform, sitk.sitkBSpline, 0.0, empty_preprocess(img2).GetPixelID())
        tmp_img = exqimg
        
        if tp2==10:
            SEQ = 'S109'
        else:
            SEQ = 'S10'+str(tp2)
#         exqimg = resample_image(exqimg)
        sitk.WriteImage(sitk.Cast(exqimg, sitk.sitkInt16), icafesavepath + 'TH_0_P'+  pi + side + '_U' + SEQ + '.tif')
        print('save to icafe path', icafesavepath + 'TH_0_P' + pi + side + '_U' + SEQ + '.tif')
        fixed_ret.append(empty_preprocess(img1))
        resampled_ret.append(empty_preprocess(tmp_img))
    return fixed_ret, resampled_ret

In [29]:
for casei in caselist:
    our_pipeline(casei)

Fixed Image 0
[384 384  75] [384, 384, 309]
save to icafe path Z:/iCafe/result/OAIMTP/0_P9404954L_U/TH_0_P9404954L_U.tif
save to icafe path Z:/iCafe/result/OAIMTP/0_P9404954L_U/TH_0_P9404954L_US100.tif
************************************************
Processing 1...
ahaknee12tp29  exist
reading image...... 9404954TP1
[384 384  75] [384, 384, 309]
Final metric value: 7153.610634321483
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 58.
Final metric value: 2546.996866561506
Optimizer's stopping condition, LBFGSBOptimizerv4: User requested
save to icafe path Z:/iCafe/result/OAIMTP/0_P9404954L_U/TH_0_P9404954L_US101.tif
************************************************
Processing 2...
ahaknee18tp29  exist
cannot find dcm path for TPid 2
